In [1]:
#define imports

import re
import ast
from IPython.display import clear_output

# define rooms and items

# Game Room
game_room = {"name": "game room", "type": "room"}
## Furniture 
couch = { "name": "couch", "type": "furniture"}
piano = { "name": "piano", "type": "furniture"}

# Bedroom 1
bedroom_1 = {"name": "bedroom 1", "type": "room"}
## Furniture 
queen_bed = {"name": "queen bed", "type": "furniture"}

# Bedroom 2
bedroom_2 = {"name": "bedroom 2", "type": "room"}
## Furniture 
double_bed = { "name": "double bed", "type": "furniture"}
dresser = {"name": "dresser", "type": "furniture"}

# Living Room
living_room = {"name": "living room", "type": "room"}
## Furniture 
dining_table = {"name": "dining table", "type": "furniture"}

# Rooms 
outside = {"name": "outside"}

# Doors
door_a = {"name": "door a", "type": "door"}
door_b = {"name": "door b", "type": "door"}
door_c = {"name": "door c", "type": "door"}
door_d = {"name": "door d", "type": "door"}
    
# Keys 
key_a = {"name": "key for door a", "type": "key", "target": door_a}
key_b = {"name": "key for door b", "type": "key", "target": door_b}
key_c = {"name": "key for door c", "type": "key", "target": door_c}
key_d = {"name": "key for door d", "type": "key", "target": door_d}

# Summary items
all_rooms = [game_room, bedroom_1, bedroom_2, living_room ,outside]
all_doors = [door_a, door_b, door_c, door_d]

#lang

es_language_file = "es.txt"
en_language_file = "en.txt"
cat_language_file = "cat.txt"

languages_availables = {"es": es_language_file, "en": en_language_file,
                        "cat": cat_language_file}

# define which items/rooms are related

object_relations = {
    # Items (doors - furnitures) each room contains 
    "game room": [couch, piano, door_a],
    "bedroom 1": [queen_bed, door_c, door_b, door_a],
    "bedroom 2": [double_bed, dresser, door_b],
    "living room": [dining_table, door_c, door_d],
    "outside": [door_a],
    
    # What key each furniture contains
    "piano": [key_a],
    "queen bed": [key_b],
    "double bed": [key_c],
    "dresser": [key_d],
    
    # Where the door belongs to
    "door a": [game_room, bedroom_1],
    "door b": [bedroom_1, bedroom_2],
    "door c": [bedroom_1, living_room],
    "door d": [living_room, outside]
}

# define game state. Do not directly change this dict. 
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This 
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside,
    
    #English per default
    "language_selected": "en"
}

In [2]:
def linebreak():
    """
    Print a line break
    """
    print("\n")

def start_game():
    """
    Start the game
    """
    choose_language()
    play_room(game_state["current_room"])

def get_locale_from(key):
    """
    Return the key localized from the language chosen.
    """
    language_selected = languages_availables.get(game_state["language_selected"])
    with open(language_selected, "r") as data:
        dictionary = ast.literal_eval(data.read())
        return dictionary[key]
        
def clear_previous_rooms(should_clear):
    if should_clear == True:
        clear_output()
    
def choose_language():
    """
    Choose a language just before playing first room
    """
    language_selection = input("Which language do you want to play the game in? Choose between: 'en' 'es' 'cat' \n").strip()
    if language_selection in languages_availables:
        language_selected = language_selection
        game_state["language_selected"] = language_selected

def print_items_collected(should_print):
    if should_print == True:
        keys_list = [get_locale_from(item["name"]) for item in game_state["keys_collected"]]
        print(get_locale_from("keys_so_far").format("\n\u2022 " + "\n\u2022 ".join(map(str, keys_list))))
        
def play_room(room, fresh_start = False):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either 
    explore (list all items in this room) or examine an item found here.
    """    
    clear_previous_rooms(fresh_start)
    
    game_state["current_room"] = room
    if(game_state["current_room"] == game_state["target_room"]):
        print(get_locale_from("congrats_finish"))
    else:
        print("\n" + get_locale_from("where_are_you").format("\033[1m" + room["name"] + "\033[0m"))
        print_items_collected(fresh_start)
        
        intended_action = input(get_locale_from("intended_action")).strip()
        if intended_action in ["explore", "explorar"]:
            explore_room(room)
            play_room(room)
        elif intended_action in ["examine", "examinar", "examina"]:
            examine_item(input(get_locale_from("what_examine")).strip())
        else:
            print(get_locale_from("not_sure"))
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    
    you_explore = get_locale_from("your_current_room").format(room["name"])
    items_found = get_locale_from("items_found").format("\n\u2022 " + "\n\u2022 ".join([item.capitalize() for item in items]))
    linebreak()
    print(you_explore + items_found)

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been 
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
    
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name or item["name"].capitalize() == item_name):
            output = get_locale_from("examine_item").format(item_name)
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += get_locale_from("unlock_with_current_key")
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += get_locale_from("locked_without_key")
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += get_locale_from("item_found").format(get_locale_from(item_found["name"]))
                else:
                    output += get_locale_from("nothing_interesting")
            print(output)
            break

    if(output is None):
        print(get_locale_from("item_not_found"))
    if(next_room and input(get_locale_from("question_next_room")).strip() in ['yes', 'si', 'sí']):
        play_room(next_room, True)
    else:
        play_room(current_room)

In [ ]:
game_state = INIT_GAME_STATE.copy()

start_game()

Which language do you want to play the game in? Choose between: 'en' 'es' 'cat' 
cat

Ara estàs a game room
